In [2]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils import flatten_list_of_dicts 

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'config', 'crowdfunding_db', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food', 'us_accidents_db']


In [5]:
# assign the us_accidents_db database to a variable name
db = mongo['us_accidents_db']

In [6]:
# List the names of the collections in the database.
# Be sure to use the variable name you assigned to your database to do this.
print(db.list_collection_names())

['us_Vision', 'Vehicle_Type', 'system.views', 'us_vehicle', 'us_race', 'us_person', 'us_drug', 'us_weather', 'us_accident']


In [7]:
# assign the collection to a variable
weather = db['us_weather']
vision = db['us_Vision']
person = db['us_person']
race = db['us_race']
drug = db['us_drug']
accident = db['us_accident']
vehicle = db['us_vehicle']
vehicle_type = db['Vehicle_Type']

In [8]:
accident.update_many(
    {},  # This empty filter selects all documents in the collection
    {'$unset': {'ACCDT_PK': ""}}  # This specifies the field to remove
)

UpdateResult({'n': 148431, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [9]:
accident_results = accident.find({})
df_accident = pd.DataFrame(list(accident_results))
df_accident.head(1)

,_id,STATE,STATENAME,ST_CASE,VE_TOTAL,PVH_INVL,PEDS,PERSONS,PERMVIT,PERNOTMVIT,...,RD_OWNERNAME,LATITUDE,LONGITUD,SP_JURNAME,HARM_EVNAME,RELJCT1NAME,WRK_ZONENAME,REL_ROADNAME,LGT_CONDNAME,FATALS
0,670f216a0b10f417d858c095,1,Alabama,10001,2,0,0,3,3,0,...,State Highway Agency,33.490967,-88.274083,No Special Jurisdiction,Motor Vehicle In-Transport,No,,On Roadway,Daylight,1


In [10]:
person.update_many(
    {},  # This empty filter selects all documents in the collection
    {'$unset': {'PERSON_PK': ""}}  # This specifies the field to remove
)

UpdateResult({'n': 362510, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [11]:
person_results = person.find({ })
df_person = pd.DataFrame(list(person_results))
df_person.head(1)

,_id,STATE,STATENAME,ST_CASE,VE_FORMS,VEH_NO,PER_NO,STR_VEH,AGE,AGENAME,...,HISPANIC,HISPANICNAME,LOCATION,LOCATIONNAME,YEAR,STATENAME_race,RACE,RACENAME,ORDER,MULTRACE
0,670f214247f7b35b50a570f1,1,Alabama,10001,2,1,1,0,34,34 Years,...,0,Not A Fatality (not Applicable),0,Occupant of a Motor Vehicle,2019,Alabama,0,Not a Fatality (not Applicable),1,0


In [12]:
# Define the aggregation pipeline with a match stage

def fn_sex_filter(state,year):
   
    sex_pipeline = [
        {
            "$match": {
                '$and': [
                    #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                    #{"SEXNAME": {'$in': ['Male', 'Female']}},
                    {"STATENAME": {'$in': state}},
                    {"YEAR": {'$in': year}},
                    {"INJ_SEV":4}
                ]
            }
        },
        {
            "$group": {
                "_id": "$SEXNAME" , "COUNT": {"$sum": 1}  # Count occurrences
            }
        },
        {
            "$sort": {
                "_id.sex": 1  # Sort by sex ascending
            }
        }
    ]
    # Execute the aggregation
    sex_plot = pd.DataFrame(list(person.aggregate(sex_pipeline))).rename(columns={'_id': 'SEX'})
    
    # Process data for the chart
    return sex_plot.to_dict(orient='records')
    


In [13]:
list_of_dict = fn_sex_filter(['New Jersey'],[2022]) ## check the data
list_of_dict


[{'SEX': 'Male', 'COUNT': 506}, {'SEX': 'Female', 'COUNT': 179}]

In [14]:
import plotly.express as px
import pandas as pd

# Assuming sex_plot is already processed and has 'Sex' and 'COUNT' columns
df_sex = pd.DataFrame(sex_plot)  # Convert sex list to a pandas DataFrame

# Define the labels and values from the DataFrame
labels = df_sex['id']  # This will be the 'Male', 'Female', etc.
values = df_sex['COUNT']  # The accident counts for each category

# Create the interactive pie chart using Plotly
fig = px.pie(
    df_sex,  # Use the DataFrame directly
    names=labels,  # Set the 'names' parameter for the labels
    values=values,  # Set the 'values' parameter for the counts
    color=labels,  # Assign colors to each category
    color_discrete_map={
        'Male': 'skyblue', 
        'Female': 'lightcoral', 
        'Reported as Unknown': 'green', 
        'Not Reported': 'orange'
    },
    title='Sex-Wise Ratio of Accidents',  # Chart title
    width=800,  # Set the width of the figure
    height=500  # Set the height of the figure
)

# Customize the hover information and appearance
fig.update_traces(textinfo='percent+label', hoverinfo='label+percent+value')

# Show the pie chart
fig.show()




NameError: name 'sex_plot' is not defined

In [21]:
def categorize_age(age):
    if 0 <= age < 5:
        return '0 to 05 Years'
    elif 5 <= age < 9:
        return '05 to 09 Years'
    elif 10 <= age < 14:
        return '10 to 14 Years'
    elif 15 <= age < 20:
        return '15 to 20 Years'
    elif 21 <= age < 24:
        return '21 to 24 Years'
    elif 25 <= age < 34:
        return '25 to 34 Years'
    elif 35 <= age < 44:
        return '35 to 44 Years'
    elif 45 <= age < 54:
        return '45 to 54 Years'   
    elif 55 <= age < 64:
        return '55 to 64 Years'
    elif 65 <= age < 74:
        return '65 to 74 Years'     
    else:
        return '75+ Age'

In [28]:
# age
# Define the aggregation pipeline with a match stage
def fn_age_filter(state,year):
    age_pipeline = [
        {
            "$match": {
                '$and': [
                    #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                    #{'AGE': {'$gte': 15, '$lte': 100}},  # Filter for age between 15 and 100
                    {"STATENAME": {'$in': state}},
                    {"YEAR": {'$in': year}},
                    {"INJ_SEV":4}
                ]
            }
        },
        {
            "$group": {
                "_id": "$AGE",  # Group by age
                "COUNT": {"$sum": 1}  # Count occurrences
            }
        },
        {
            "$sort": {
                "_id": 1  # Sort by age ascending
            }
        }
    ]

    # Execute the aggregation
    df_age_plot = pd.DataFrame(list(person.aggregate(age_pipeline))).rename(columns={'_id': 'AGE'})
    df_age_plot['AGE_GROUP'] = df_age_plot['AGE'].apply(categorize_age)
    
    df_age_agg_plot = df_age_plot.groupby('AGE_GROUP', as_index=False)['COUNT'].sum()
    
    #df_age_agg_plot.head(20)
    # Convert df_age_agg_plot DataFrame to a list of dictionaries
    age_agg_list = df_age_agg_plot.to_dict(orient='records')
    
    # Display the list of dictionaries
    print(age_agg_list)

In [30]:
list_of_dict1 = fn_age_filter(['Texas'],[2021]) ## check the data
list_of_dict1

[{'AGE_GROUP': '0 to 05 Years', 'COUNT': 47}, {'AGE_GROUP': '05 to 09 Years', 'COUNT': 38}, {'AGE_GROUP': '10 to 14 Years', 'COUNT': 36}, {'AGE_GROUP': '15 to 20 Years', 'COUNT': 291}, {'AGE_GROUP': '21 to 24 Years', 'COUNT': 322}, {'AGE_GROUP': '25 to 34 Years', 'COUNT': 873}, {'AGE_GROUP': '35 to 44 Years', 'COUNT': 661}, {'AGE_GROUP': '45 to 54 Years', 'COUNT': 523}, {'AGE_GROUP': '55 to 64 Years', 'COUNT': 545}, {'AGE_GROUP': '65 to 74 Years', 'COUNT': 336}, {'AGE_GROUP': '75+ Age', 'COUNT': 828}]


In [ ]:
# Increase figure size
plt.figure(figsize=(12, 6))

# Create the bar chart with adjusted bar width
plt.bar(df_age_agg_plot['AGE_GROUP'], df_age_agg_plot['COUNT'], color='skyblue', width=0.8)

# Set title and labels
plt.title('Count of Occurrences by Age group in Texas (2022)')
plt.xlabel('Age Group')
plt.ylabel('Count')

# Rotate x-tick labels for better readability
plt.xticks(df_age_agg_plot['AGE_GROUP'], rotation=45)

# Add grid for y-axis
plt.grid(axis='y')

# Save the plot
plt.savefig('age_agg_plot.png', bbox_inches='tight')  # Use bbox_inches to fit the plot tightly
plt.show()

In [31]:
# person_type
# Define the aggregation pipeline with a match stage
def fn_person_filter(state,year):
    person_type_pipeline = [
        {
            "$match": {
                '$and': [
                    {"PER_TYP": {"$in": [1, 2, 5]}}, 
                   
                    {"STATENAME": {'$in': state}},
                    {"YEAR": {'$in': year}},
                    {"INJ_SEV":4}
                ]
            }
        },
        {
            "$group": {
                "_id": "$PER_TYPNAME", 
                "COUNT": {"$sum": 1}  # Count occurrences
            }
        },
        {
            "$sort": {
                "_id": 1  
            }
        }
    ]
    
    # Execute the aggregation
    person_plot = list(person.aggregate(person_type_pipeline))
     # Process data for the chart
    return flatten_list_of_dicts(person_plot)



#df_person_plot.head(20)

In [32]:
list_of_dict2 = fn_person_filter(['Texas'],[2022]) ## check the data
list_of_dict2

[{'_id': 'Driver of a Motor Vehicle In-Transport', 'COUNT': 2696},
 {'_id': 'Passenger of a Motor Vehicle In-Transport', 'COUNT': 786},
 {'_id': 'Pedestrian', 'COUNT': 797}]

In [ ]:
plt.figure(figsize=(10,2))
plt.barh(df_person_plot['PER_TYPNAME'], df_person_plot['COUNT'], color='lightgreen')
plt.title('Count of PER_TYP by Name in Texas (2022)')
plt.xlabel('Count')
plt.ylabel('PER_TYP Name')
plt.tight_layout()

# Save the plot
plt.savefig('per_typ_count.png')
plt.show()

In [36]:
# person_type
# Define the aggregation pipeline with a match stage
def fn_person_drinking_filter(state,year):
    person_drinking_pipeline = [
        {
            "$match": {
                '$and': [
                    {"DRINKING": {"$in": [0, 1, 8, 9]}}, 
                    {"PER_TYP": 1},
                    {"STATENAME": {'$in': state}},
                    {"YEAR": {'$in': year}},
                    {"INJ_SEV":4}
                ]
            }
        },
        {
            "$group": {
                "_id": "$DRINKINGNAME", 
                "COUNT": {"$sum": 1}  # Count occurrences
            }
        },
        {
            "$sort": {
                "_id": 1  
                
            }
        }
    ]

    # Execute the aggregation
    person_drinking_plot = list(person.aggregate(person_drinking_pipeline))
    # Process data for the chart
    return flatten_list_of_dicts(person_drinking_plot)

#df_person_drinking_plot.head(20)

In [37]:
list_of_dict3 = fn_person_drinking_filter(['Florida'],[2022]) ## check the data
list_of_dict3

[{'_id': 'No (Alcohol Not Involved)', 'COUNT': 1051},
 {'_id': 'Not Reported', 'COUNT': 6},
 {'_id': 'Reported as Unknown', 'COUNT': 644},
 {'_id': 'Yes (Alcohol Involved)', 'COUNT': 321}]

In [ ]:
# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(df_person_drinking_plot['DRINKINGNAME'], df_person_drinking_plot['COUNT'], color='skyblue')
plt.title('Count of Drinking Status in Texas (2022)')
plt.xlabel('Drinking Status')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Show the plot
plt.tight_layout()
plt.savefig('per_drinking_count.png')
plt.show()

In [38]:
# person_drug
# Define the aggregation pipeline with a match stage
def fn_person_drugs_filter(state,year):
    dr_drugs_pipeline = [
        {
            "$match": {
                '$and': [
                    {"DRUGS": {"$in": [0, 1, 8, 9]}}, 
                    {"PER_TYP": 1},
                    {"STATENAME": {'$in': state}},
                    {"YEAR": {'$in': year}},
                    {"INJ_SEV":4}
                ]
            }
        },
        {
            "$group": {
                "_id": "$DRUGSNAME", 
                "COUNT": {"$sum": 1}  # Count occurrences
            }
        },
        {
            "$sort": {
                "_id": 1  
                
            }
        }
    ]

    # Execute the aggregation
    dr_drugs_plot = list(person.aggregate(dr_drugs_pipeline))
    # Process data for the chart
    
    return  flatten_list_of_dicts(dr_drugs_plot)                    


#df_person_drugs_plot.head(20)

In [39]:
list_of_dict4 = fn_person_drugs_filter(['Florida'],[2022]) ## check the data
list_of_dict4

[{'_id': 'No (drugs not involved)', 'COUNT': 1099},
 {'_id': 'Not Reported', 'COUNT': 6},
 {'_id': 'Reported as Unknown', 'COUNT': 675},
 {'_id': 'Yes (drugs involved)', 'COUNT': 242}]

In [ ]:

# vehicle type
# Define the aggregation pipeline with a match stage

vehicle_pipeline = [
    {
        "$match": {
            '$and': [
                #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                {"STATENAME": 'New Jersey'},
                {"YEAR": {'$in': [2022]}},
                {"DEATHS": {"$gt": 0}}
            ]
        }
    },
    {
        "$group": {
            "_id": "$BODY_TYP",  # Group by age
            "COUNT": {"$sum": 1}  # Count occurrences
        }
    },
    {
        "$sort": {
            "_id": 1  # Sort by age ascending
        }
    }
]

# Execute the aggregation
df_vehicle = pd.DataFrame(list(vehicle.aggregate(vehicle_pipeline))).rename(columns={'_id': 'BODY_TYP'})
df_vehicle_type = pd.DataFrame(list(vehicle_type.find( {} )))

# Merging DataFrames
merged_df = pd.merge(df_vehicle, df_vehicle_type, left_on='BODY_TYP', right_on='Code', how='inner')
df_vehicle_plot = merged_df.groupby('Classification')['COUNT'].sum().reset_index()

# Plot a pie chart
df_vehicle_plot.plot.pie(y='COUNT', labels = df_vehicle_plot['Classification'], autopct='%1.2f%%', figsize=(8, 5), colors=['skyblue', 'lightcoral','green'])

# Set the title
plt.title('Vehicle type wise ratio')
# Save the plot
plt.savefig('vehicle_typ_count.png')
# Show the plot
plt.show()

In [ ]:

# drunk driver
# Define the aggregation pipeline with a match stage

drunk_driver_pipeline = [
    {
        "$match": {
            '$and': [
                #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                {"STATENAME": 'Texas'},
                {"YEAR": {'$in': [2022]}},
                {"INJ_SEVNAME":4}
            ]
        }
    },
    {
        "$group": {
            "_id": "$DR_DRINKNAME",  # Group by age
            "COUNT": {"$sum": 1}  # Count occurrences
        }
    },
    {
        "$sort": {
            "_id": 1  # Sort by age ascending
        }
    }
]

# Execute the aggregation
df_drunk_driver_plot = pd.DataFrame(list(vehicle.aggregate(drunk_driver_pipeline))).rename(columns={'_id': 'DR_DRINKNAME'})

# Plot a pie chart
df_drunk_plot.plot.pie(y='COUNT', labels=df_drunk_plot['DR_DRINKNAME'], autopct='%1.1f%%', figsize=(8, 5), colors=['skyblue', 'lightcoral'])

# Set the title
plt.title('Drunk driver Ratio')

# Show the plot
plt.show()

In [ ]:
# race type
# Define the aggregation pipeline with a match stage

race_pipeline = [
    {
        "$match": {
            '$and': [
                {"STATENAME": 'Texas'},
                {"YEAR": {'$in': [2022]}},
                {"INJ_SEV":4}
            ]
        }
    },
    {
        "$group": {
            "_id": "$RACENAME",  # Group by race
            "COUNT": {"$sum": 1}  # Count occurrences
        }
    },
    {
        "$sort": {
            "_id": 1  # Sort by race ascending
        }
    }
]

# Execute the aggregation
df_race_plot = pd.DataFrame(list(person.aggregate(race_pipeline))).rename(columns={'_id': 'RACENAME'})

plt.figure(figsize=(10,4))
plt.barh(df_race_plot['RACENAME'], df_race_plot['COUNT'], color='lightgreen')
plt.title('Race of that person who died in fatal accident')
plt.xlabel('Count')
plt.ylabel('Race Name')
plt.tight_layout()

# Save the plot
plt.savefig('race.png')
plt.show()

In [ ]:
#Get the sex count per year or for selected state per year 
def getSexFilter(year,state):
    #Calculate fatalities by sex
    if year and state:
        sex_pipeline = [
            {
                "$match": {
                    '$and': [
                        #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                        #{"SEXNAME": {'$in': ['Male', 'Female']}},
                        {"STATENAME": {'$in': state}},
                        {"YEAR": {'$in': year}},
                        {"INJ_SEV":4}
                    ]
                }
            },
            {
                "$group": {
                    "_id": "$SEXNAME" , "COUNT": {"$sum": 1}  # Count occurrences
                }
            },
            {
                "$sort": {
                    "_id.sex": 1  # Sort by sex ascending
                }
            }
        ]
    elif year:
        sex_pipeline = [
            {
                "$match": {
                    '$and': [
                        #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                        #{"SEXNAME": {'$in': ['Male', 'Female']}},
                        #{"STATENAME": {'$in': state}},
                        {"YEAR": {'$in': year}},
                        {"INJ_SEV":4}
                    ]
                }
            },
            {
                "$group": {
                    "_id": "$SEXNAME" , "COUNT": {"$sum": 1}  # Count occurrences
                }
            },
            {
                "$sort": {
                    "_id.sex": 1  # Sort by sex ascending
                }
            }
        ]
    else:
        # sex Types total in 4 years 
        sex_pipeline = [
            {
                "$match": {
                    '$and': [
                        #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                        #{"SEXNAME": {'$in': ['Male', 'Female']}},
                        #{"STATENAME": {'$in': state}},
                        {"YEAR": {'$in': year}},
                        {"INJ_SEV":4}
                    ]
                }
            },
            {
                "$group": {
                    "_id": "$SEXNAME" , "COUNT": {"$sum": 1}  # Count occurrences
                }
            },
            {
                "$sort": {
                    "_id.sex": 1  # Sort by sex ascending
                }
            }
        ]

      # Execute the aggregation
    sex_plot = list(person.aggregate(sex_pipeline))
    print(sex_plot)

In [20]:
sex_pipeline = [{'$match': {'$and': [{'SEXNAME': {'$in': ['Male', 'Female']}}, {'YEAR': {'$in':[2020]}}, {'INJ_SEV': 4}]}}, {'$group': {'_id': '$SEXNAME', 'COUNT': {'$sum': 1}}}, {'$sort': {'_id.sex': 1}}]

# Execute the aggregation
sex_plot = list(person.aggregate(sex_pipeline))
print(sex_plot)


[{'_id': 'Female', 'COUNT': 10746}, {'_id': 'Male', 'COUNT': 28161}]


In [26]:
def getageFilter(year,state):
        age_pipeline = [
        {
            "$match": {
                '$and': [
                    #{"PER_TYP": 1},  # Filter for PER_TYP = 1
                    #{'AGE': {'$gte': 15, '$lte': 100}},  # Filter for age between 15 and 100
                    {"STATENAME": {'$in': state}},
                    {"YEAR": {'$in': year}},
                    {"INJ_SEV":4}
                ]
            }
        },
        {
            "$group": {
                "_id": "$AGE",  # Group by age
                "COUNT": {"$sum": 1}  # Count occurrences
            }
        },
        {
            "$sort": {
                "_id": 1  # Sort by age ascending
            }
        }
    ]
    # Execute the aggregation
    df_age_plot = pd.DataFrame(list(person.aggregate(age_pipeline))).rename(columns={'_id': 'AGE'})
    df_age_plot['AGE_GROUP'] = df_age_plot['AGE'].apply(categorize_age)
    
    df_age_agg_plot = df_age_plot.groupby('AGE_GROUP', as_index=False)['COUNT'].sum()
    
    #df_age_agg_plot.head(20)
    # Convert df_age_agg_plot DataFrame to a list of dictionaries
    age_agg_list = df_age_agg_plot.to_dict(orient='records')
    
    # Display the list of dictionaries
    print(age_agg_list)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)